In [1]:
import numpy as np
from scipy.spatial.distance import cdist
import time
from sklearn.preprocessing import LabelBinarizer
import random as ran
import time
import torch

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [3]:
"""
Superclass for classes of Preconditioners.

"""
class Preconditioner(object):

    def __init__(self, name = ""):
        self.name = name

In [4]:
class InducingPointsHelper(object):

	def __init__(self, seed):
		ran.seed(seed)
		self.name = "InducingPointsHelper"

	"""
	Returns a random selection of points from the given dataset
		X - Dataset
		M - Number of points to be selected
	"""
	def get_random_inducing_points(self, X, M):
		rand = ran.sample(range(0, X.shape[0]), M)
		return X[rand]

In [5]:
class RBF(object):

    def __init__(self, lengthscale=1., variance=1., noise=1.):
        super(RBF, self).__init__()
        self.lengthscale = lengthscale
        self.variance = variance
        self.jitter = 1e-9
        # self.noise = noise / self.variance + self.jitter# dividing by variance for new strategy
        self.noise = noise

    def K(self, X1, X2):
        """ GP squared exponential kernel """
        pairwise_dists = cdist(X1, X2, 'sqeuclidean')
        print(type(pairwise_dists[0,0]))
        return self.variance*np.exp(-0.5 * pairwise_dists / self.lengthscale ** 2)
        # return pairwise_dists

In [6]:
"""
Solve linear system using conjugate gradient
Params:
    K - Covariance Matrix
    Y - Target labels
    init - Initial solution
    thershold - Termintion criteria
"""
class PlainCG(object):

    def __init__(self, K, Y, init=None, threshold=1e-9):
        N = np.shape(K)[0]
        if init is None:
            init = np.zeros((N,1))

        self.K = K
        self.Y = Y.flatten()

        x = init
        r = Y - np.dot(K, x) #initialise residual gradient
        p = r

        t = 0
        while True:
            alpha = np.dot(r.T, r) / np.dot(p.T, np.dot(K, p))
            x = x + alpha*p
            r_prev = r
            r = r - alpha*np.dot(K, p)

            if ((np.dot(r.T,r).flatten() < (threshold*N)) or (t>15000)):
                break
            beta = np.dot(r.T, r) / np.dot(r_prev.T, r_prev)
            p = r + beta*p
            t = t + 1

        self.iterations = t
        self.result = x

In [7]:
torch.cuda.device_count()

2

In [7]:
"""
Solve linear system using conjugate gradient
Params:
    K - Covariance Matrix
    Y - Target labels
    init - Initial solution
    thershold - Termintion criteria
"""
def MultiCGGPU(K, Y, init=None, tol=1e-5, atol=1e-9, max_iterations=15000, cuda=False, num_gpus=3):
    N = np.shape(K)[0]
    if init is None:
        init = np.zeros((N,1))

    # self.K = K
    # self.Y = Y.flatten()


    x = init
    R = Y - np.dot(K, x) #initialise residual gradient

    # Move two kernel splits to 2 GPUs
    K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
    split_size = K.shape[0] // num_gpus
    if cuda:
        Ks = []
        for i in range(num_gpus):
            if i < (num_gpus-1):
                Ks.append(
                    K[i*split_size:(i+1)*split_size].to('cuda:' + str(i))
                )
            else:
                Ks.append(
                    K[i*split_size:].to('cuda:' + str(i))
                )

    iterations = []
    solutions = []

    for dim in range(Y.shape[1]):
        print('Starting CG for dimension {}'.format(dim))
        since = time.time()
        # get current residual vector
        r = R[:, dim][:, None]

        p = r

        t = 0

        x = torch.from_numpy(init).type(torch.FloatTensor) #.cuda()
        r = torch.from_numpy(r).type(torch.FloatTensor) #.cuda()
        p = torch.from_numpy(p).type(torch.FloatTensor) #.cuda()

        if cuda:
            x = x.to('cuda:0')
            r = r.to('cuda:0')
            ps = [p.to('cuda:' + str(i)) for i in range(num_gpus)]

        while True:
            with torch.no_grad():
                # alpha = np.dot(r.T, r) / np.dot(p.T, np.dot(K, p))
                Kps = [Ks[i].mm(ps[i]).to('cuda:0') for i in range(num_gpus)]
                Kp = torch.cat(Kps, dim=0)
                pKp = ps[0].t().mm(Kp)

                alpha = r.t().mm(r) / pKp
                x = x + alpha*ps[0]
                r_prev = r
                # r = r - alpha*np.dot(K, p)
                r = r - alpha * Kp

                # if ((np.dot(r.T,r).flatten() < (threshold*N)) or (t>15000)):
                if ((r.t().mm(r).item() <= max(tol*np.linalg.norm(Y[:, dim]), atol*N)) or (t>max_iterations)):
                    break
                # if ((r.t().mm(r).item() < (threshold*N)) or (t>max_iterations)):
                #     break
                # beta = np.dot(r.T, r) / np.dot(r_prev.T, r_prev)
                beta = r.t().mm(r) / r_prev.t().mm(r_prev)
                ps[0] = r + beta*ps[0]

                # we need to send the updated p to gpu_i (two vector transfers in total)
                ps = [ps[0].to('cuda:' + str(i)) for i in range(num_gpus)]

                t = t + 1

        print('Iterations needed: {}'.format(t))
        print('Time elapsed: {}'.format(time.time() - since))
        iterations.append(t)
        if cuda:
            x = x.cpu()
        solutions.append(x.numpy())

    return np.hstack(solutions), iterations

In [8]:
"""
Solve linear system using conjugate gradient
Params:
    K - Covariance Matrix
    Y - Target labels
    init - Initial solution
    thershold - Termintion criteria
"""
class PlainCGGPU(object):

    def __init__(self, K, Y, init=None, threshold=1e-9, cuda=False):
        N = np.shape(K)[0]
        if init is None:
            init = np.zeros((N,1))

        self.K = K
        self.Y = Y.flatten()

        x = init
        r = Y - np.dot(K, x) #initialise residual gradient
        p = r

        t = 0
        
        x = torch.from_numpy(x).type(torch.FloatTensor) #.cuda()
        r = torch.from_numpy(r).type(torch.FloatTensor) #.cuda()
        p = torch.from_numpy(p).type(torch.FloatTensor) #.cuda()
        K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
        
        if cuda:
            x = x.cuda()
            r = r.cuda()
            p = p.cuda()
            K = K.cuda()

        while True:
            with torch.no_grad():
                # alpha = np.dot(r.T, r) / np.dot(p.T, np.dot(K, p))
                alpha = r.t().mm(r) / p.t().mm(K).mm(p)
                x = x + alpha*p
                r_prev = r
                # r = r - alpha*np.dot(K, p)
                r = r - alpha * K.mm(p)

                # if ((np.dot(r.T,r).flatten() < (threshold*N)) or (t>15000)):
                if ((r.t().mm(r).item() < (threshold*N)) or (t>15000)):
                    break
                # beta = np.dot(r.T, r) / np.dot(r_prev.T, r_prev)
                beta = r.t().mm(r) / r_prev.t().mm(r_prev)
                p = r + beta*p
                t = t + 1

        self.iterations = t
        if cuda:
            x = x.cpu()
        self.result = x.numpy()

In [9]:
class Cg(object):

    def __init__(self, K, Y, init=None, threshold=1e-9, cuda=False):
        N = np.shape(K)[0]
        if init is None:
            init = np.zeros((N,1))

        self.K = K
        self.Y = Y.flatten()

        x = init
        r = Y - np.dot(K, x) #initialise residual gradient
        p = r
        
        t = 0
        
#         x = torch.from_numpy(x).type(torch.FloatTensor) #.cuda()
#         r = torch.from_numpy(r).type(torch.FloatTensor) #.cuda()
#         p = torch.from_numpy(p).type(torch.FloatTensor) #.cuda()
#         K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
        
#         if cuda:
#             x = torch.from_numpy(x).cuda()
#             r = torch.from_numpy(r).cuda()
#             p = torch.from_numpy(p).cuda()
#             K = torch.from_numpy(K).cuda()
        
        with torch.no_grad():
            while True:
                # scalar
                # old_resid_norm = r.norm()**2 # r.t().mm(r).item()
                old_resid_norm = r.T @ r
                # alpha = old_resid_norm / p.t().mm(K).mm(p)
                alpha = old_resid_norm / p.T @ K @ p
                
                # vector
                x = x + alpha*p
                # r_prev = r
                # vector
                # r = r - alpha * K.mm(p)
                r = r - alpha * K @ p
                
                #print('Iteration:', t)
                #print('Shape r', r.shape)
                
                # resid_norm = r.norm()**2 # r.t().mm(r).item()
                resid_norm = r.T @ r
                
                break

                if (resid_norm < (threshold*N)) or (t>15000):
                    break
                beta = resid_norm / old_resid_norm
                p = r + beta*p
                t = t + 1
        del K, p, r

        self.iterations = t
        self.result = x
        
#         if cuda:
#             for i in range(1, 10):
#                 try:
#                     torch.cuda.empty_cache()
#                     self.result = x.cpu().numpy()
#                     break
#                 except RuntimeError as e:
#                     print('Attempt {}. Out of memory. Retrying...'.format(i))
#                     print(e)
#                     # torch.cuda.empty_cache()
#                     time.sleep(i * 2)
#         else:
#             self.result = x.numpy()

In [10]:
class RegularPcgPyTorch(object):

    def __init__(self, K, Y, P, init=None, threshold=1e-9, preconInv=None):
        N = np.shape(K)[0]
        if init is None:
            init = np.zeros((N,1))

        if preconInv is None:
            preconInv = np.linalg.inv(P)

        self.K = K
        self.P = P
        self.Y = Y.flatten()

        x = init
        r = Y - np.dot(K, x) #initialise residual gradient
        z = np.dot(preconInv, r)
        p = z

        outerC = 0
        
        # move data to pytorch / cuda
        x = torch.from_numpy(x).type(torch.FloatTensor) #.cuda()
        r = torch.from_numpy(r).type(torch.FloatTensor) #.cuda()
        z = torch.from_numpy(z).type(torch.FloatTensor) #.cuda()
        p = torch.from_numpy(p).type(torch.FloatTensor) #.cuda()
        K = torch.from_numpy(K).type(torch.FloatTensor) #.cuda()
        preconInv = torch.from_numpy(preconInv).type(torch.FloatTensor) #.cuda()
        
        with torch.no_grad():

            while True:
                alpha = r.t().mm(z) / p.t().mm(K).mm(p)
                x = x + alpha*p
                r_prev = r
                r = r - alpha*K.mm(p)
                # norm(residual) <= max(tol*norm(b), atol) might also be an option
                if r.t().mm(r) < threshold*N or outerC>10000:
                    break
                z_prev = z
                z = preconInv.mm(r)
                beta = z.t().mm(r) / z_prev.t().mm(r_prev)
                p = z + beta*p
                outerC = outerC + 1

        self.iterations = outerC
        self.result = x.cpu().numpy()

In [11]:
class RegularPcg(object):

	def __init__(self, K, Y, P, init=None, threshold=1e-9, preconInv=None):
		N = np.shape(K)[0]
		if init is None:
			init = np.zeros((N,1))

		if preconInv is None:
			preconInv = np.linalg.inv(P)

		self.K = K
		self.P = P
		self.Y = Y.flatten()

		x = init
		r = Y - np.dot(K, x) #initialise residual gradient
		z = np.dot(preconInv, r)
		p = z

		outerC = 0

		while True:
			alpha = np.dot(r.T, z) / np.dot(p.T,np.dot(K, p))
			x = x + alpha*p
			r_prev = r
			r = r - alpha*np.dot(K,p)
			# norm(residual) <= max(tol*norm(b), atol) might also be an option
			if (np.dot(r.T, r).flatten() < threshold*N or outerC>10000):
				break
			z_prev = z
			z = np.dot(preconInv, r)
			beta = np.dot(z.T, r) / np.dot(z_prev.T, r_prev)
			p = z + beta*p
			outerC = outerC + 1
			if outerC % 10 == 0:
				print('{} iterations completed.'.format(outerC))
		
		self.iterations = outerC
		self.result = x

In [12]:
"""
Nystrom Preconditioner
"""
class Nystrom(Preconditioner):

	"""
	Construct preconditioning matrix
		X - Training data
		kern - Class of kernel function
		Xm - Inducing points
		addNoise - Flag indicating whether to add likelihood variance to kernel matrix
	"""
	def __init__(self, X, kern, Xm, addNoise=True):
		super(Nystrom, self).__init__("Nystrom")

		start = time.time()

		self.kern = kern
		self.X = X
		N = np.shape(X)[0]
		M = np.shape(Xm)[0]
		self.M = M
		self.N = N

		Kxm = kern.K(X, Xm)
		Km = kern.K(Xm, Xm)

		self.Kxm = Kxm
		self.Km = Km + 1e-6*np.identity(M) # jitter
		self.KmInv = np.linalg.inv(self.Km)
        
		print('Type:', type(Kxm[0,0]))
		print('Size:', Kxm.shape, self.KmInv.shape)

		if addNoise:
			self.precon = np.dot(np.dot(Kxm,self.KmInv),Kxm.T) + self.kern.noise*np.identity(N)
		else:
			self.precon = np.dot(np.dot(Kxm,self.KmInv),Kxm.T)

		self.duration = time.time() - start

	"""
	Compute inversion of the preconditioner.
	"""
	def get_inversion(self):
		N = np.shape(self.X)[0]
		M = np.shape(self.Km)[0]
		noise = self.kern.noise
		inv_noise = float(1) / noise
		noise_matrix = noise*np.identity(M)

		eigs, eigv = np.linalg.eig(self.KmInv)
		for i in range(len(eigv)):
			if (eigs[i] < self.kern.jitter):
				eigs[i] = self.kern.jitter
			eigs[i] = np.sqrt(eigs[i])

		eigsD = np.diag(eigs)
		left = np.dot(self.Kxm, np.dot(eigv, eigsD))
		right = np.dot(eigsD, np.dot(eigv.T, self.Kxm.T))

		return inv_noise*self.woodbury_inversion(np.identity(N), left, noise_matrix, right)

	"""
	Implementation of Woodbury's matrix inversion lemma.
	"""
	def woodbury_inversion(self, Ainv, U, Cinv, V):
		left_outer = np.dot(Ainv, U)
		right_outer = np.dot(V, Ainv)
		inner = np.linalg.inv(Cinv + np.dot(V, np.dot(Ainv, U)))
		return Ainv - np.dot(left_outer, np.dot(inner, right_outer))

In [13]:
# (train_data, train_labels), (test_data, test_labels) = FashionMNIST()

train_data = np.load('../../datasets/export/fashion_mnist/numpy/train_data_fashion_mnist.npy').astype('float32')
test_data = np.load('../../datasets/export/fashion_mnist/numpy/test_data_fashion_mnist.npy').astype('float32')
train_labels = np.load('../../datasets/export/fashion_mnist/numpy/train_targets_fashion_mnist.npy').astype('float32')
test_labels = np.load('../../datasets/export/fashion_mnist/numpy/test_targets_fashion_mnist.npy').astype('float32')

train_data = train_data[:60000]
train_labels = train_labels[:60000]

# Convert one-hot to integers
train_labels = np.argmax(train_labels, axis=1)
test_labels = np.argmax(test_labels, axis=1)

D = train_data[0].reshape(-1).shape[0]
N = len(train_data)

# Flatten the images
train_data = train_data.reshape(-1, D)
test_data = test_data.reshape(-1, D)

In [14]:
def threshold_binarize(data, threshold):
    data_bin = np.where(data>threshold, 1, 0)
    return data_bin

threshold = 10

train_data_bin = threshold_binarize(train_data, threshold).astype('float32')
test_data_bin = threshold_binarize(test_data, threshold).astype('float32')

In [15]:
active_pixels_train = train_data_bin.sum(axis=1, keepdims=True)
active_pixels_test = test_data_bin.sum(axis=1, keepdims=True)

In [16]:
label_binarizer = LabelBinarizer(pos_label=1, neg_label=-1)
train_labels_bin = label_binarizer.fit_transform(train_labels).astype('float32')
test_labels_bin = label_binarizer.fit_transform(test_labels).astype('float32')

In [24]:
# M = int(np.sqrt(N))
# ipHelper = InducingPointsHelper(0)
# XmRandom = ipHelper.get_random_inducing_points(train_data_bin,M)

In [13]:
# XmRandom.shape

(100, 784)

In [50]:
# kernel = RBF(lengthscale=np.sqrt(D/2), variance=1., noise=1e-6)

In [26]:
# jitter = 1e-6
# K = kernel.K(train_data_bin, train_data_bin) + jitter*np.identity(N)

In [51]:
# 5K estimates
kernel_var = 0.1789339259732666
lengthscale = 7.177267570983543
jitter = 0.04144659858222422
gamma = 1. / (2*lengthscale**2)

In [59]:
gamma

0.00970625574163014

In [36]:
jitter

0.04

In [44]:
# jitter = 1e-6
# kernel_var = 100.0
K = 1 * rbf_kernel(train_data_bin, Y=train_data_bin, gamma=None) + 1e-6*np.identity(N).astype('float32')

In [17]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import polynomial_kernel

In [52]:
# jitter = 1e-6
# kernel_var = 100.0
K = kernel_var * rbf_kernel(train_data_bin, Y=train_data_bin, gamma=gamma) + jitter*np.identity(N).astype('float32')

In [18]:
def opu_kernel(x, y, gamma=1):
    kernel = polynomial_kernel(x, Y=y, degree=2, gamma=1, coef0=0)
    norm_x_sq = np.linalg.norm(x, ord=2, axis=1, keepdims=True) ** 2
    norm_y_sq = np.linalg.norm(y, ord=2, axis=1, keepdims=True) ** 2

    # corresponds to element-wise addition of norm_x^2 * norm_y^2
    kernel += np.dot(norm_x_sq, norm_y_sq.T)
    
    kernel *= gamma
    
    return kernel

In [21]:
0.001**2

1e-06

In [19]:
# we chose gamma according to the random feature optimization step
gamma = 0.001
# alpha is chosen like that as well
alpha = 10
K = opu_kernel(train_data_bin, y=train_data_bin, gamma=gamma) + alpha*np.identity(N).astype('float32')

In [20]:
K.shape

(60000, 60000)

In [25]:
# import bunch

In [26]:
# kern = bunch.Bunch()

In [27]:
# kern.K = K

In [28]:
# kern.noise = jitter
# kern.jitter = 1e-9

In [64]:
# originally adding noise
# K = kernel.K(train_data_bin,train_data_bin) + kernel.noise*np.identity(N).astype('float32')

<class 'numpy.float64'>


In [24]:
type(K[0,0])

numpy.float32

In [54]:
# kernel.noise

1e-06

In [21]:
K

array([[338.05    , 252.16599 , 172.07997 , ..., 160.11401 , 273.526   ,
         90.945   ],
       [252.16599 , 427.698   , 239.27599 , ..., 231.814   , 409.691   ,
         88.257996],
       [172.07997 , 239.27599 , 209.71199 , ..., 171.628   , 253.436   ,
         60.128   ],
       ...,
       [160.11401 , 231.814   , 171.628   , ..., 200.962   , 246.89102 ,
         58.15    ],
       [273.526   , 409.691   , 253.436   , ..., 246.89102 , 490.20004 ,
         96.266014],
       [ 90.945   ,  88.257996,  60.128   , ...,  58.15    ,  96.266014,
         67.122   ]], dtype=float32)

In [20]:
type(K[0,0])

numpy.float32

In [147]:
# prec = Nystrom(train_data_bin, kern, XmRandom)

Type: <class 'numpy.float32'>
Size: (10000, 100) (100, 100)


In [102]:
# prec.precon.shape

(10000, 10000)

In [148]:
# inv = prec.get_inversion()

In [81]:
# inv.shape

(5000, 5000)

In [26]:
# train_labels[:, None].shape

(5000, 1)

In [ ]:
# 1K dimensions: 1 second/label
# 10K dimensiosn: 1724 seconds/label (0.5 hours)
# in theory: 1000x slower

In [60]:
from scipy import linalg

In [ ]:
dual_coef = linalg.solve(K, y, sym_pos=True, overwrite_a=False)

In [22]:
since = time.time()
# print('Running CG for dim', dim)
# pcg = RegularPcgPyTorch(K, train_labels_bin[:, dim][:, None], prec.precon, threshold=1e-9, preconInv=inv)
# dual_coef = linalg.solve(K, train_labels_bin, sym_pos=True, overwrite_a=False)
dual_coef, iterations = MultiCGGPU(K, train_labels_bin, tol=1e-5, atol=1e-9, max_iterations=10*N, cuda=True, num_gpus=2)
# dual_cofs.append(pcg.result)
# coef, info = cg(K, train_labels_bin[:, dim], tol=1e-5) # M=inv
# coef, info = gmres(K, train_labels_bin[:, dim], tol=1e-5)
# dual_cofs.append(coef.reshape((-1, 1)))
# print('Info:', info)
print('Done. Iterations:', iterations)
print('Time:', time.time() - since)
    
# dual_coef = np.hstack(dual_cofs)

Starting CG for dimension 0
Iterations needed: 749
Time elapsed: 13.044814825057983
Starting CG for dimension 1
Iterations needed: 725
Time elapsed: 12.123180150985718
Starting CG for dimension 2
Iterations needed: 762
Time elapsed: 12.656646013259888
Starting CG for dimension 3
Iterations needed: 739
Time elapsed: 12.257280349731445
Starting CG for dimension 4
Iterations needed: 780
Time elapsed: 12.91981053352356
Starting CG for dimension 5
Iterations needed: 737
Time elapsed: 12.25653624534607
Starting CG for dimension 6
Iterations needed: 772
Time elapsed: 12.79292368888855
Starting CG for dimension 7
Iterations needed: 733
Time elapsed: 12.154973983764648
Starting CG for dimension 8
Iterations needed: 720
Time elapsed: 11.930649757385254
Starting CG for dimension 9
Iterations needed: 734
Time elapsed: 12.18154764175415
Done. Iterations: [749, 725, 762, 739, 780, 737, 772, 733, 720, 734]
Time: 253.08103275299072


In [23]:
K_test = opu_kernel(test_data_bin, y=train_data_bin, gamma=gamma)
prediction = np.dot(K_test, dual_coef)

In [76]:
# K_test = kernel_var * rbf_kernel(test_data_bin, train_data_bin)
# prediction = np.dot(K_test, dual_coef)

In [106]:
type(prediction[0,0])

numpy.float32

In [107]:
prediction.shape

(10000, 10)

In [24]:
score = np.sum(np.equal(np.argmax(prediction, 1), np.argmax(test_labels_bin, 1))) / len(test_data) * 100

In [43]:
# 10K
score

86.21

In [25]:
# OPU: 60K!!!
score

88.55

In [31]:
score

83.81

In [68]:
score

83.81

In [38]:
K_test = kernel_var * rbf_kernel(test_data_bin, train_data_bin)
prediction = np.dot(K_test, dual_coef)

In [36]:
# kernel_var 1
score

83.81

In [39]:
# kernel_var 10
score

83.81

In [44]:
# kernel_var 100
score

83.98